In [ ]:
! gdown 1yEjicXasMGFaALQobQWgsSxQgLEtxruC
! unzip /content/ThaiDepression.zip

Downloading...
From: https://drive.google.com/uc?id=1yEjicXasMGFaALQobQWgsSxQgLEtxruC
To: /content/ThaiDepression.zip
100% 41.1k/41.1k [00:00<00:00, 44.2MB/s]
Archive:  /content/ThaiDepression.zip
  inflating: ThaiDepression/AHB_01.csv  
  inflating: ThaiDepression/AHB_02.csv  
  inflating: ThaiDepression/AHB_03.csv  
  inflating: ThaiDepression/AHB_04.csv  
  inflating: ThaiDepression/AHB_05.csv  
  inflating: ThaiDepression/AHB_06.csv  
  inflating: ThaiDepression/AHB_07.csv  
  inflating: ThaiDepression/AHB_08.csv  
  inflating: ThaiDepression/AHB_09.csv  
  inflating: ThaiDepression/AHB_10.csv  
  inflating: ThaiDepression/AHB_11.csv  
  inflating: ThaiDepression/AHB_12.csv  
  inflating: ThaiDepression/AHB_13.csv  
  inflating: ThaiDepression/All_AHB.csv  


In [ ]:
import pandas as pd
import numpy as np
import os
from numpy import array
from math import sqrt
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GRU, BatchNormalization
from pandas import read_csv
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, SimpleRNN
import json
import psutil 
from datetime import datetime, timezone, timedelta

In [ ]:
file_list = os.listdir('ThaiDepression')
file_list.sort()
df_list = []
for i in range(14):
    df_list.append(pd.read_csv('ThaiDepression/' + file_list[i], index_col=0))

In [ ]:
df_list

[         Chiang_Rai    Nan  Phayao  Phrae  Chiang_Mai  Mae_Hong_Son  Lampang  \
 2016-10       10383   5315    4266   4057       15408          2002     9011   
 2016-11       10383   5315    4265   4057       15406          2002     9011   
 2016-12       10382   5315    4265   4057       15404          2002     9011   
 2017-01       10382   5655    4265   4057       15393          2002     9011   
 2017-02       10382   5655    4264   4057       15392          2002     9011   
 ...             ...    ...     ...    ...         ...           ...      ...   
 2022-03       25223   9711   10788   7988       47292          6924    17444   
 2022-04       25456   9781   10932   8095       48020          7004    17606   
 2022-05       25619   9833   10977   8122       48155          7026    17703   
 2022-06       25836   9884   11069   8161       48616          7090    17783   
 2022-07       26174  10019   11163   8250       49096          7169    17951   
 
          Lamphun   AHB_1 

In [ ]:
# split a univariate sequence into samples
def uni_split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


    # split a multivariate sequence into samples
def multi_split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

In [ ]:
%matplotlib inline
# define scope of configs
AHB_list = [4, 5, 6, 7]
LSTM_hidden_units = [8, 16, 32, 64, 128]
n_layer = [1, 2, 3]
n_batch = [2, 4, 8, 16, 32, 64, 128]
n_steps = [5]
patience_list = [3, 5, 7, 10]
mul_uni = ['uni','mul']
dropouts = [0.2, 0.3, 0.5]


timezone_offset = 7.0  # Bangkok Time (UTC+07:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
count = 1
performance_json = []
for g in AHB_list:
    for h in patience_list:
        for a in n_steps:
            for b in dropouts:
                for c in LSTM_hidden_units:
                    for d in n_layer:
                        for e in n_batch:
                            for f in mul_uni:

                                print('PROGRESS: [ ' + str(count) + ' / ' + str(len(dropouts)*len(AHB_list)*len(n_steps)*len(LSTM_hidden_units)*len(n_layer)*len(n_batch)*len(patience_list)*len(mul_uni)) + ' ]')
                                
                                if f == 'uni':
                                    if g == 13:
                                        raw_seq = list(df_list[g]['Country_Level'])
                                        ahb = 'Country_Level'
                                    else:    
                                        raw_seq = list(df_list[g]['AHB_'+str(g+1)])
                                        ahb = 'AHB_'+str(g+1)
                                    n_features = 1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq).reshape(len(raw_seq),-1))
                                    X, y = uni_split_sequence(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
                                if f == 'mul':
                                    if g == 13:
                                        raw_seq = list(df_list[13].drop(df_list[13].columns[:77], axis=1).values)
                                        ahb = 'Country_Level'
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[13]['Country_Level']).reshape(len(df_list[13]['Country_Level']),-1))
                                    else:    
                                        raw_seq = list(df_list[g].drop(['Country_Level'], axis=1).values)
                                        ahb = 'AHB_'+str(g+1)
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[g]['AHB_'+str(g+1)]).reshape(len(df_list[13]['AHB_'+str(g+1)]),-1))
                                    n_features = len(raw_seq[0])-1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq))
                                    X, y = multi_split_sequences(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

                                callback = EarlyStopping(monitor='loss', patience=h)
                                model = Sequential()
                                if d == 1:
                                    model.add(LSTM(units=c, return_sequences=False, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                if d == 2:
                                    model.add(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                    model.add(LSTM(units=c, return_sequences=False, activation='tanh'))
                                    model.add(Dropout(b))
                                if d == 3:
                                    model.add(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                    model.add(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh'))
                                    model.add(Dropout(b))
                                    model.add(LSTM(units=c, return_sequences=False, activation='tanh'))
                                    model.add(Dropout(b))
                                model.add(Dense(units=1, kernel_initializer='he_uniform', activation='linear'))
                                model.compile(loss='mse', optimizer='adam')  
                                history = model.fit(X_train, y_train, epochs=2000, batch_size=e, verbose=0, callbacks=[callback])
                                last_epoch = len(history.history['loss']) 

                                yhat = model.predict(X_test, verbose=0)
                                yhat = yhat.reshape(yhat.shape[0],-1)
                                mae = mean_absolute_error(y_test, yhat)
                                mse = mean_squared_error(y_test, yhat)
                                rmse = mean_squared_error(y_test, yhat, squared = False)
                                mape = mean_absolute_percentage_error(y_test, yhat)
                                
                                if f == 'uni':
                                    yhat_inverse = scaler.inverse_transform(yhat)
                                    y_test_inverse = scaler.inverse_transform(y_test)
                                if f == 'mul':
                                    yhat_inverse = scaler_y.inverse_transform(yhat)
                                    y_test_inverse = scaler_y.inverse_transform(y_test.reshape(len(y_test),-1))


                                mae_inverse = mean_absolute_error(y_test_inverse, yhat_inverse)
                                mse_inverse = mean_squared_error(y_test_inverse, yhat_inverse)
                                rmse_inverse = mean_squared_error(y_test_inverse, yhat_inverse, squared = False)
                                mape_inverse = mean_absolute_percentage_error(y_test_inverse, yhat_inverse)

                                performance_json.append({'model': 'LSTM', 'AHB_number': ahb, 'Multi_Uni': f, \
                                                         'n_layer': d, 'LSTM_hidden_units': c, 'dropouts': b,'n_batch': e,\
                                                         'n_steps': a, 'last_epoch': last_epoch, 'patience_list': h, 'MAE_normalize': float(mae), \
                                                         'MSE_normalize': float(mse), 'RMSE_normalize': float(rmse), 'MAPE_normalize': float(mape), \
                                                         'MAE': float(mae_inverse), 'MSE': float(mse_inverse), 'RMSE': float(rmse_inverse), 'MAPE': float(mape_inverse)})
                                

                                json_formatted_str = json.dumps(performance_json[-1], indent=2)
                                print(json_formatted_str)
                                print('Percent CPU Usage: ' + str(psutil.cpu_percent()))
                                print('Percent Ram Usage: ' + str(psutil.virtual_memory()[2]))
                                print(datetime.now(tzinfo))

                                count = count + 1

PROGRESS: [ 1 / 10080 ]
{
  "model": "LSTM",
  "AHB_number": "AHB_5",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "LSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 12,
  "patience_list": 3,
  "MAE_normalize": 0.11428863995354531,
  "MSE_normalize": 0.013160205834662252,
  "RMSE_normalize": 0.11471794033481533,
  "MAPE_normalize": 0.11977962190750562,
  "MAE": 4915.897321428572,
  "MSE": 24347935.118704658,
  "RMSE": 4934.362686173834,
  "MAPE": 0.05844972714934678
}
Percent CPU Usage: 6.9
Percent Ram Usage: 2.8
2022-10-11 19:26:58.059967+07:00
PROGRESS: [ 2 / 10080 ]
{
  "model": "LSTM",
  "AHB_number": "AHB_5",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "LSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 5,
  "last_epoch": 16,
  "patience_list": 3,
  "MAE_normalize": 0.12416309451995591,
  "MSE_normalize": 0.015531519871357447,
  "RMSE_normalize": 0.12462551853997418,
  "MAPE_normalize": 0.13013613564362567,
  "MAE": 5340.627232142

{
  "model": "LSTM",
  "AHB_number": "AHB_5",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "LSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 8,
  "n_steps": 5,
  "last_epoch": 32,
  "patience_list": 3,
  "MAE_normalize": 0.2161930918006217,
  "MSE_normalize": 0.04713699668989835,
  "RMSE_normalize": 0.21711056328492712,
  "MAPE_normalize": 0.22651821929804225,
  "MAE": 9299.113839285714,
  "MSE": 87209020.94677734,
  "RMSE": 9338.577030082117,
  "MAPE": 0.11055126074196554
}
Percent CPU Usage: 16.3
Percent Ram Usage: 3.2
2022-10-11 19:27:15.390879+07:00
PROGRESS: [ 6 / 10080 ]


Streaming output truncated to the last 5000 lines.
  "MAE": 3071.989955357141,
  "MSE": 9878585.481140122,
  "RMSE": 3143.0217118467576,
  "MAPE": 0.029859926011607724
}
Percent CPU Usage: 19.0
Percent Ram Usage: 3.6
2022-10-12 13:54:55.883890+07:00
PROGRESS: [ 9873 / 10080 ]
{
  "model": "LSTM",
  "AHB_number": "AHB_8",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "LSTM_hidden_units": 8,
  "dropouts": 0.5,
  "n_batch": 4,
  "n_steps": 5,
  "last_epoch": 37,
  "patience_list": 10,
  "MAE_normalize": 0.0810397916639776,
  "MSE_normalize": 0.006613475041757079,
  "RMSE_normalize": 0.08132327490796887,
  "MAPE_normalize": 0.08475903281827703,
  "MAE": 3109.982142857141,
  "MSE": 9739773.727068206,
  "RMSE": 3120.8610553929193,
  "MAPE": 0.03027258345907152
}
Percent CPU Usage: 17.2
Percent Ram Usage: 3.6
2022-10-12 13:55:00.635229+07:00
PROGRESS: [ 9874 / 10080 ]
{
  "model": "LSTM",
  "AHB_number": "AHB_8",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "LSTM_hidden_units": 8,
  "dropouts": 0.5,
  "

In [ ]:
name = "all_performance_LSTM_5nsteps_AHB5-6-7-8"
json.dump(performance_json, open(name + ".json","w"))
df_json = pd.read_json('/content/' + name + '.json')
df_json.to_csv('/content/' + name + '.csv', index=False)
#! cp /content/all_performance_LSTM_5nsteps_AHB5-6-7-8.json /content/drive/MyDrive/Depression
! cp /content/all_performance_LSTM_5nsteps_AHB5-6-7-8.csv /content/drive/MyDrive/Depression

In [ ]:
df_json

,model,AHB_number,Multi_Uni,n_layer,LSTM_hidden_units,dropouts,n_batch,n_steps,last_epoch,patience_list,MAE_normalize,MSE_normalize,RMSE_normalize,MAPE_normalize,MAE,MSE,RMSE,MAPE
0,LSTM,AHB_5,uni,1,8,0.2,2,5,12,3,0.114289,0.013160,0.114718,0.119780,4915.897321,2.434794e+07,4934.362686,0.058450
1,LSTM,AHB_5,mul,1,8,0.2,2,5,16,3,0.124163,0.015532,0.124626,0.130136,5340.627232,2.873515e+07,5360.517443,0.063502
2,LSTM,AHB_5,uni,1,8,0.2,4,5,19,3,0.117842,0.013981,0.118242,0.123522,5068.745536,2.586699e+07,5085.959769,0.060271
3,LSTM,AHB_5,mul,1,8,0.2,4,5,17,3,0.084911,0.007357,0.085773,0.088833,3652.271205,1.361122e+07,3689.338696,0.043388
4,LSTM,AHB_5,uni,1,8,0.2,8,5,32,3,0.216193,0.047137,0.217111,0.226518,9299.113839,8.720902e+07,9338.577030,0.110551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,LSTM,AHB_8,mul,3,128,0.5,32,5,79,10,0.013896,0.000364,0.019071,0.014200,533.260045,5.356305e+05,731.867817,0.005148
10076,LSTM,AHB_8,uni,3,128,0.5,64,5,55,10,0.006722,0.000055,0.007392,0.007078,257.954241,8.048020e+04,283.690324,0.002517
10077,LSTM,AHB_8,mul,3,128,0.5,64,5,50,10,0.054967,0.003122,0.055875,0.057325,2109.430804,4.597866e+06,2144.263444,0.020512
10078,LSTM,AHB_8,uni,3,128,0.5,128,5,51,10,0.008751,0.000086,0.009261,0.009163,335.823661,1.263096e+05,355.400581,0.003270
